In [2]:
sc


''

In [3]:
orderItems = sc.textFile("retail_db/order_items")

NameError: name 'sc' is not defined

In [59]:
orderItemsMap = orderItems.map( lambda oi: (float(oi.split(",")[1]) ,  float(oi.split(",")[4])))

orderItemsMap = orderItems.map( lambda a: a.split(",")[4])

In [60]:
reducedOrderItems = orderItemsMap.reduceByKey(lambda curr , next: curr + next)


In [202]:
for s in reducedOrderItems.take(1): print(s)

(32768.0, 1029.94)


In [63]:
l = range(1,10000)
lRDD = sc.parallelize(l)
type(lRDD)

pyspark.rdd.RDD

In [201]:
productsRaw = open("/home/zubairidrees/hadoop-2.7.3/data/retail_db/products/part-00000").read().splitlines()
type(productsRaw)
productsRDD = sc.parallelize(productsRaw)
productsRDD.take(1)

['1,2,Quest Q64 10 FT. x 10 FT. Slant Leg Instant U,,59.98,http://images.acmesports.sports/Quest+Q64+10+FT.+x+10+FT.+Slant+Leg+Instant+Up+Canopy']

In [200]:
sqlContext
help(sqlContext)

In [199]:
#spark 1.6
#sqlContext.load("/retail_db_json/order_items","json").show()

#spark 2
spark.read.load("/user/zubairidrees/retail_db/json/retail_db_json/order_items", format="json").show()

#sqlContext.read.json("/user/zubairidrees/retail_db/json/retail_db_json/order_items/part-r-00000-6b83977e-3f20-404b-9b5f-29376ab1419e").show()
spark.read.json("/user/zubairidrees/retail_db/json/retail_db_json/order_items").show()


## String Manipulation


In [87]:
orders = sc.textFile("retail_db/retail_db/orders")
orders.take(1)

[u'1,2013-07-25 00:00:00.0,11599,CLOSED']

In [102]:
s = orders.first()
type(s)
s[2:13]
len(s)
print("Concat String with int" + str(1))




Concat String with int1


u'20130725'

## Convert Date YYYY-MM-DD into YYYYMMDD



In [103]:
s.split(",")[1].split(" ")[0].replace("-","")

u'20130725'

## map 

In [203]:
orders = sc.textFile("retail_db/retail_db/orders")
orders.map(lambda o: o.split(",")[1].split(" ")[0].replace("-","")).take(10)
orders.map(lambda o: o.split(",")[1].split(" ")[0].replace("-","")).count()

orderItems = sc.textFile("retail_db/retail_db/order_items")
orderItems.first()
orderItems.map(lambda oi:( int(oi.split(",")[1]),float(oi.split(",")[5]) )).take(1)


[(1, 299.98)]

## flatMap

In [136]:
linesList = ["How are you","where are you","How is the things going"]
linesRDD = sc.parallelize(linesList)

linesMap = linesRDD.map(lambda l :l.split(" ")).collect()
for i in linesMap: print i

linesFlatMap = linesRDD.flatMap(lambda l :l.split(" ")).collect()
for i in linesFlatMap: print i



['How', 'are', 'you']
['where', 'are', 'you']
['How', 'is', 'the', 'things', 'going']
How
are
you
where
are
you
How
is
the
things
going


## filter

In [204]:
orders = sc.textFile("retail_db/retail_db/orders")

ordersCompleted = orders.filter(lambda o: o.split(",")[3] == "COMPLETE" or o.split(",")[3] == "CLOSED").take(1)
for i in ordersCompleted: print i
    
    
#filter records of year 2014

orders2014 = orders.filter(lambda o: 
                           (o.split(",")[3] in [ "COMPLETE", "CLOSED"]) and
                           o.split(",")[1].split("-")[0] == "2014").take(1)
for i in orders2014: print i


1,2013-07-25 00:00:00.0,11599,CLOSED
25882,2014-01-01 00:00:00.0,4598,COMPLETE


## Join

In [179]:
orders = sc.textFile("retail_db/retail_db/orders")
orderItems = sc.textFile("retail_db/retail_db/order_items")

ordersMap = orders.map(lambda o: (int(o.split(",")[0]) ,o.split(",")[3] ))

orderItemsMap = orderItems.map(lambda oi: (int(oi.split(",")[1]) ,float(oi.split(",")[4]) ))

ordersJoin = ordersMap.join(orderItemsMap)


ordersLeftoin = ordersMap.leftOuterJoin(orderItemsMap)

orderOuterLeftJoinFilter = ordersLeftoin.filter(lambda o: o[1][1] == None)
print orderOuterLeftJoinFilter.count()
for i in orderOuterLeftJoinFilter.take(1): print i


11452
(43692, (u'PENDING_PAYMENT', None))


## Reduce

In [198]:
orderItems = sc.textFile("retail_db/retail_db/order_items")
orderItemsFiltered = orderItems.filter(lambda oi: int(oi.split(",")[1]) == 2)


orderItemsMap = orderItemsFiltered.map(lambda oi: float(oi.split(",")[4])) 
#for i in orderItemsMap.take(3): print i  
orderItemsReduced = orderItemsMap.reduce(lambda curr , next : curr + next)
print orderItemsReduced

## using operator
from operator import add
orderItemsReduced = orderItemsMap.reduce(add)
print orderItemsReduced

##using if else in reduce
##Find provided order id order item detail having minimum sub total

orderItemsReduced = orderItemsMap.reduce(lambda curr , next : next if(curr > next) else curr)
print orderItemsReduced

 


579.98
579.98
129.99


## ReduceByKey / CountByKey / AggregateByKey

# Count By Status

In [210]:
orders = sc.textFile("retail_db/retail_db/orders")
for i in orders.take(1):print i
ordersStatusMap = orders.map(lambda o: (o.split(",")[3], 1) )
for i in ordersStatusMap.take(1): print i
    
orderStatusCount = ordersStatusMap.countByKey();
print orderStatusCount


1,2013-07-25 00:00:00.0,11599,CLOSED
(u'CLOSED', 1)
defaultdict(<type 'int'>, {u'COMPLETE': 22899, u'PAYMENT_REVIEW': 729, u'PROCESSING': 8275, u'CANCELED': 1428, u'PENDING': 7610, u'CLOSED': 7556, u'PENDING_PAYMENT': 15030, u'SUSPECTED_FRAUD': 1558, u'ON_HOLD': 3798})


#### groupByKey don't use combiner & thats why iit is slow compare to reduceByKey & aggregateByKey
#### groupByKey
#### sum of (1-1000) -> 1 + 2 + 3 + 4 + 5 + 6 ......

#### reduceByKey & aggregateByKey
#### sum of (1-1000) -> sum(1,25) + sum(26,50)...

#### in case of average we can't use reduceByKey & aggregateByKey in place of groupByKey as result will be
#### different so if result of reduceByKey/aggregateByKey is same then only use it

#### reduceByKey take one function & aggregateByKey take 2 functions one is for computing the intermidiate value & other for final value

#### if reducer & combinor  is same then use reduceByKey




In [229]:
#Get Revenue for each order_id

orderItems = sc.textFile("retail_db/retail_db/order_items")
for i in orderItems.take(1):print i

orderItemsMap = orderItems.map(lambda oi: (int(oi.split(",")[1]), float(oi.split(",")[4])) )
for i in orderItemsMap.take(3):print i
    
orderItemsGroupByKey = orderItemsMap.groupByKey();
print orderItemsGroupByKey
for i in orderItemsGroupByKey.take(5): print i
l = orderItemsGroupByKey.first()
print l[0]
list(l[1])


    

1,1,957,1,299.98,299.98
(1, 299.98)
(2, 199.99)
(2, 250.0)
PythonRDD[940] at RDD at PythonRDD.scala:48
(2, <pyspark.resultiterable.ResultIterable object at 0x7fec4c59f950>)
(4, <pyspark.resultiterable.ResultIterable object at 0x7fec4c54c210>)
(8, <pyspark.resultiterable.ResultIterable object at 0x7fec4c54c990>)
(10, <pyspark.resultiterable.ResultIterable object at 0x7fec4c54c8d0>)
(12, <pyspark.resultiterable.ResultIterable object at 0x7fec4c54cf50>)
2


TypeError: 'ResultIterable' object is not callable